# Genesis probability of simulated tropical cyclones



In [ ]:
%matplotlib inline

import os
from os import walk
from os.path import join as pjoin
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy.feature as feature
import shapely.geometry as sg

from statsmodels.nonparametric.kernel_density import KDEMultivariate

from Utilities import track
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_context("poster")
sns.set_style("whitegrid")

In [ ]:
def readTracks(trackFile):
    """
    Read a track file and create a `GeoPandas.GeoDataFrame` from the data, with
    separate polyline features for each unique track in the file. Also adds a
    nominal intensity category based on the central pressure value.
    
    :param str trackFile: Path to a TCRM-format track file
    
    :returns: `GeoPandas.GeoDataFrame` of tracks
    """
    tracks = track.ncReadTrackData(trackFile)
    trackgdf = []
    for t in tracks:
        gdf = pd.DataFrame.from_records(t.data)
        gdf['TrackId'] = "{0:03d}-{1:05d}".format(*t.trackId)
        trackgdf.append(gdf)
    trackgdf = pd.concat(trackgdf)
    return trackgdf.groupby('TrackId').first()

In [ ]:
filelist = []
datapath = "C:/WorkSpace/data/tcha/tracks"
for (dirpath, dirnames, filenames) in walk(datapath):
    filelist.extend([fn for fn in filenames if fn.endswith('nc')])
    break
nfiles = len(filelist)
print(f"There are {nfiles} track files")

In [ ]:
genesislist = []
for sim, f in enumerate(filelist):
    tracks = readTracks(pjoin(datapath,f))

    genesislist.append(tracks)
    if sim > 100: break
genesisdf = pd.concat(genesislist)

In [ ]:
genesisdf[['Longitude', 'Latitude']].values

In [ ]:
dens = KDEMultivariate(data=genesisdf[['Longitude', 'Latitude']].values, var_type='cc', bw='cv_ml')
x = np.arange(90,180, 0.1)
y = np.arange(-35, -5, 0.1)
xx, yy = np.meshgrid(x, y)
xy = np.vstack([xx.ravel(), yy.ravel()])
pdf = dens.pdf(data_predict=xy)
pdf = pdf.reshape(xx.shape)


In [ ]:
fig = plt.figure(figsize=(12,5))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0))
ax.coastlines(resolution='10m', color='black', linewidth=1)
ax.add_feature(feature.BORDERS)
cf = ax.contourf(xx, yy, pdf, cmap='magma_r')
cs = ax.contour(xx, yy, pdf, cmap='magma_r')
plt.colorbar(cf, format='%.1e')